In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import optuna

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [2]:
train = pd.read_parquet('../src/data/raw/train.parquet')
train['monthyear'] = (pd.to_datetime(train['S_2']).dt.year.astype(str)+
                      pd.to_datetime(train['S_2']).dt.month.map("{:02}".format).astype(str))
x = train.groupby("customer_ID")['monthyear'].agg(['nunique'])

## Exp Set

In [8]:
prefixes = ['oof_v2_dpv25fold_seed42', 'oof_v2_5fold_seed98', 
            'oof_v4_dpv25fold_seed42', 'oof_lgbm_baseline_5fold_seed42',
            'oof_v1_dpv35fold_seed42', 'cboof_v2_dpv25fold_seed802'
           ]

valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
    
for i in range(0, len(valids)):
    for j in range(i+1, len(valids)):
        print(f'{prefixes[i]} and {prefixes[j]} corr: ', valids[i]['prediction'].corr(valids[j]['prediction']))

oof_v2_dpv25fold_seed42 and oof_v2_5fold_seed98 corr:  0.9965551637703891
oof_v2_dpv25fold_seed42 and oof_v4_dpv25fold_seed42 corr:  0.9978415655159716
oof_v2_dpv25fold_seed42 and oof_lgbm_baseline_5fold_seed42 corr:  0.9973313158075049
oof_v2_dpv25fold_seed42 and oof_v1_dpv35fold_seed42 corr:  0.9942262830511286
oof_v2_dpv25fold_seed42 and cboof_v2_dpv25fold_seed802 corr:  0.9257227351885323
oof_v2_5fold_seed98 and oof_v4_dpv25fold_seed42 corr:  0.9965033053333487
oof_v2_5fold_seed98 and oof_lgbm_baseline_5fold_seed42 corr:  0.9961773434270609
oof_v2_5fold_seed98 and oof_v1_dpv35fold_seed42 corr:  0.993663208502304
oof_v2_5fold_seed98 and cboof_v2_dpv25fold_seed802 corr:  0.9213241450500348
oof_v4_dpv25fold_seed42 and oof_lgbm_baseline_5fold_seed42 corr:  0.9975730574774834
oof_v4_dpv25fold_seed42 and oof_v1_dpv35fold_seed42 corr:  0.9944807588297289
oof_v4_dpv25fold_seed42 and cboof_v2_dpv25fold_seed802 corr:  0.9288144901145169
oof_lgbm_baseline_5fold_seed42 and oof_v1_dpv35fold_see

In [11]:
prefixes = ['oof_v2_dpv25fold_seed42', 'oof_v1_dpv35fold_seed42', 
#             'oof_v4_dpv25fold_seed42',
#             'oof_lgbm_baseline_5fold_seed42',
            'cboof_v2_dpv25fold_seed802',
           ]

valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_1 = trial.suggest_uniform('s1', 0.0, 1.0)
    p_2 = trial.suggest_uniform('s2', 0.0, 1.0)
    p_3 = trial.suggest_uniform('s3', 0.0, 1.0)
#     p_4 = trial.suggest_uniform('s4', 0.0, 1.0)
#     p_5 = trial.suggest_uniform('s5', 0.0, 1.0)
#     p_6 = trial.suggest_uniform('s6', 0.0, 1.0)
    
    y = valids[0]['target'].values
    
    a_1 = valids[0].drop(['target','customer_ID'], axis=1).values
    a_2 = valids[1].drop(['target','customer_ID'], axis=1).values
    a_3 = valids[2].drop(['target','customer_ID'], axis=1).values
#     a_4 = valids[3].drop(['target','customer_ID'], axis=1).values
#     a_5 = valids[4].drop(['target','customer_ID'], axis=1).values
#     a_6 = valids[5].drop(['target','customer_ID'], axis=1).values
    
    a_4 = a_5 = a_6 = 0
    p_4 = p_5 = p_6 = 0
    
    a = (p_1 * a_1 + p_2 * a_2 + p_3 * a_3 + p_4 * a_4 + p_5 * a_5 + p_6 * a_6)
    b = a.reshape(len(a_1))/(p_1+p_2+p_3+p_4+p_5+p_6)

    return amex_metric(y, np.clip(b, 0, 1))

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=600)

[I 2022-08-16 08:47:55,317] A new study created in memory with name: no-name-fed3642e-9900-4645-bc13-746dcc42aca0
[I 2022-08-16 08:47:55,453] Trial 0 finished with value: 0.7732058846933854 and parameters: {'s1': 0.5813764241248154, 's2': 0.742224666204605, 's3': 0.2475214167321862}. Best is trial 0 with value: 0.7732058846933854.
[I 2022-08-16 08:47:55,584] Trial 1 finished with value: 0.7662383355326621 and parameters: {'s1': 0.4769052633821639, 's2': 0.5110134332551899, 's3': 0.2826447415912611}. Best is trial 0 with value: 0.7732058846933854.
[I 2022-08-16 08:47:55,713] Trial 2 finished with value: 0.754271469352205 and parameters: {'s1': 0.6465220492137151, 's2': 0.7595234814458293, 's3': 0.7773951449644937}. Best is trial 0 with value: 0.7732058846933854.
[I 2022-08-16 08:47:55,846] Trial 3 finished with value: 0.7418353250363247 and parameters: {'s1': 0.11348549977663813, 's2': 0.5621216315850913, 's3': 0.7551564166475533}. Best is trial 0 with value: 0.7732058846933854.
[I 2022

[I 2022-08-16 08:48:00,664] Trial 35 finished with value: 0.7948042882186406 and parameters: {'s1': 0.2627105524372389, 's2': 0.5957286171674344, 's3': 0.005080695933922324}. Best is trial 35 with value: 0.7948042882186406.
[I 2022-08-16 08:48:00,827] Trial 36 finished with value: 0.7944910517402597 and parameters: {'s1': 0.24103841751874666, 's2': 0.5672222112661869, 's3': 0.005421804952057725}. Best is trial 35 with value: 0.7948042882186406.
[I 2022-08-16 08:48:00,970] Trial 37 finished with value: 0.7633899491354228 and parameters: {'s1': 0.07570224349448634, 's2': 0.5951955543637969, 's3': 0.2173915347385297}. Best is trial 35 with value: 0.7948042882186406.
[I 2022-08-16 08:48:01,114] Trial 38 finished with value: 0.7778786267016763 and parameters: {'s1': 0.14412197312324482, 's2': 0.4723563202400419, 's3': 0.07912000996250602}. Best is trial 35 with value: 0.7948042882186406.
[I 2022-08-16 08:48:01,248] Trial 39 finished with value: 0.7579649056750395 and parameters: {'s1': 0.23

[I 2022-08-16 08:48:05,973] Trial 70 finished with value: 0.7920101099377288 and parameters: {'s1': 0.954758813153645, 's2': 0.3320565114524289, 's3': 0.036103424285521914}. Best is trial 69 with value: 0.7979190687842872.
[I 2022-08-16 08:48:06,139] Trial 71 finished with value: 0.7971072093380649 and parameters: {'s1': 0.9983384449111072, 's2': 0.3753334124576896, 's3': 0.006717657016878343}. Best is trial 69 with value: 0.7979190687842872.
[I 2022-08-16 08:48:06,288] Trial 72 finished with value: 0.7874736286304005 and parameters: {'s1': 0.9914313054864986, 's2': 0.15411533344998252, 's3': 0.06906121986078873}. Best is trial 69 with value: 0.7979190687842872.
[I 2022-08-16 08:48:06,444] Trial 73 finished with value: 0.7930726594928873 and parameters: {'s1': 0.8381083707824776, 's2': 0.3763030318015597, 's3': 0.0263202542318208}. Best is trial 69 with value: 0.7979190687842872.
[I 2022-08-16 08:48:06,591] Trial 74 finished with value: 0.7843733558990451 and parameters: {'s1': 0.88982

[I 2022-08-16 08:48:11,600] Trial 105 finished with value: 0.7856166074192759 and parameters: {'s1': 0.7890968093651769, 's2': 0.3378422232756508, 's3': 0.07981757851318068}. Best is trial 87 with value: 0.7984070112723578.
[I 2022-08-16 08:48:11,799] Trial 106 finished with value: 0.798311824419734 and parameters: {'s1': 0.8578362715170849, 's2': 0.3010790135837873, 's3': 0.00025260709421808154}. Best is trial 87 with value: 0.7984070112723578.
[I 2022-08-16 08:48:11,955] Trial 107 finished with value: 0.7894456291873065 and parameters: {'s1': 0.8535519404151553, 's2': 0.17343615950180813, 's3': 0.047139578214571924}. Best is trial 87 with value: 0.7984070112723578.
[I 2022-08-16 08:48:12,113] Trial 108 finished with value: 0.7929498314312835 and parameters: {'s1': 0.9141364830703983, 's2': 0.3147147904944061, 's3': 0.028263492076664534}. Best is trial 87 with value: 0.7984070112723578.
[I 2022-08-16 08:48:12,263] Trial 109 finished with value: 0.7867579549243007 and parameters: {'s1'

[I 2022-08-16 08:48:17,467] Trial 140 finished with value: 0.7836545723238901 and parameters: {'s1': 0.8839293965573574, 's2': 0.19876738606960553, 's3': 0.09620758644786778}. Best is trial 132 with value: 0.7984518053560541.
[I 2022-08-16 08:48:17,658] Trial 141 finished with value: 0.7982768915226628 and parameters: {'s1': 0.9577282499031624, 's2': 0.16460428697643104, 's3': 0.00041990051161961924}. Best is trial 132 with value: 0.7984518053560541.
[I 2022-08-16 08:48:17,880] Trial 142 finished with value: 0.7975324400914734 and parameters: {'s1': 0.9595729679189289, 's2': 0.16996093163318765, 's3': 0.0033052503618866656}. Best is trial 132 with value: 0.7984518053560541.
[I 2022-08-16 08:48:18,034] Trial 143 finished with value: 0.790068184834241 and parameters: {'s1': 0.977273936002971, 's2': 0.16078083247167127, 's3': 0.047448156685192484}. Best is trial 132 with value: 0.7984518053560541.
[I 2022-08-16 08:48:18,167] Trial 144 finished with value: 0.7483279646140086 and parameters

...

[I 2022-08-16 08:57:24,732] Trial 3058 finished with value: 0.789808882743952 and parameters: {'s1': 0.8638075754424113, 's2': 0.21656031681874954, 's3': 0.04651352213576097}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:24,920] Trial 3059 finished with value: 0.7942986293048502 and parameters: {'s1': 0.8829502248842801, 's2': 0.16889537048570408, 's3': 0.01894863949050181}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:25,099] Trial 3060 finished with value: 0.7832611903255331 and parameters: {'s1': 0.2079334976657622, 's2': 0.14125185369468163, 's3': 0.030922567987776033}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:25,327] Trial 3061 finished with value: 0.7984049295057333 and parameters: {'s1': 0.8325336954825878, 's2': 0.19642358709114893, 's3': 0.00046428252142575045}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:25,577] Trial 3062 finished with value: 0.7981871376582588 and pa

[I 2022-08-16 08:57:31,826] Trial 3092 finished with value: 0.7981218218607562 and parameters: {'s1': 0.7382846947075805, 's2': 0.2858113791326663, 's3': 0.001293923384286122}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:32,058] Trial 3093 finished with value: 0.7981546699535205 and parameters: {'s1': 0.6704643165516401, 's2': 0.30880678156386415, 's3': 0.00015529608751597203}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:32,285] Trial 3094 finished with value: 0.7880638221786909 and parameters: {'s1': 0.8071489378734255, 's2': 0.253989082156001, 's3': 0.0569506957725607}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:32,475] Trial 3095 finished with value: 0.7944193723655633 and parameters: {'s1': 0.8549288138072588, 's2': 0.17625794631587613, 's3': 0.01728882651389561}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:32,664] Trial 3096 finished with value: 0.7935710176473602 and param

[I 2022-08-16 08:57:39,081] Trial 3126 finished with value: 0.7897438113504542 and parameters: {'s1': 0.8915271919644338, 's2': 0.16629645486495337, 's3': 0.046084291836941994}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:39,271] Trial 3127 finished with value: 0.7936527629696339 and parameters: {'s1': 0.7876743470661512, 's2': 0.12344932864779107, 's3': 0.01908877583268094}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:39,482] Trial 3128 finished with value: 0.7980952118756734 and parameters: {'s1': 0.5858394275318501, 's2': 0.24030647403430375, 's3': 0.0008529689473837622}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:39,703] Trial 3129 finished with value: 0.7875757144090946 and parameters: {'s1': 0.8539196719143859, 's2': 0.195760110182864, 's3': 0.06059520801150388}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:39,901] Trial 3130 finished with value: 0.7915626280943366 and para

[I 2022-08-16 08:57:46,096] Trial 3160 finished with value: 0.783988150206828 and parameters: {'s1': 0.7582752237128733, 's2': 0.23016103748790545, 's3': 0.08482027042666807}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:46,322] Trial 3161 finished with value: 0.798217795185196 and parameters: {'s1': 0.906793452652247, 's2': 0.15657905545426803, 's3': 0.000549292270359994}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:46,549] Trial 3162 finished with value: 0.7896161498993908 and parameters: {'s1': 0.8105804109916441, 's2': 0.1995318870835768, 's3': 0.04487055477493586}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:46,720] Trial 3163 finished with value: 0.7626342905972314 and parameters: {'s1': 0.6987498544129875, 's2': 0.25891090670873373, 's3': 0.3474183797373791}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:46,948] Trial 3164 finished with value: 0.7983593216647442 and parameter

[I 2022-08-16 08:57:52,953] Trial 3194 finished with value: 0.7916484132435047 and parameters: {'s1': 0.7314403970069084, 's2': 0.22384134149560828, 's3': 0.029700199255783176}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:53,143] Trial 3195 finished with value: 0.7941522787721225 and parameters: {'s1': 0.863095712448199, 's2': 0.1638899702688824, 's3': 0.019111778352846623}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:53,361] Trial 3196 finished with value: 0.7982257922476482 and parameters: {'s1': 0.6537104869099578, 's2': 0.2616189153557952, 's3': 0.0005433238834204608}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:53,578] Trial 3197 finished with value: 0.7982299259726358 and parameters: {'s1': 0.7867834818495846, 's2': 0.3132132689486757, 's3': 0.0006657133978945354}. Best is trial 1838 with value: 0.7985375360974156.
[I 2022-08-16 08:57:53,808] Trial 3198 finished with value: 0.7984123375364105 and par

In [12]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

number     value             datetime_start          datetime_complete  \
1838    1838  0.798538 2022-08-16 08:53:23.909544 2022-08-16 08:53:24.138186   
3105    3105  0.798525 2022-08-16 08:57:34.331053 2022-08-16 08:57:34.574063   
2179    2179  0.798520 2022-08-16 08:54:29.318536 2022-08-16 08:54:29.536954   
1089    1089  0.798517 2022-08-16 08:51:03.401757 2022-08-16 08:51:03.616844   
2033    2033  0.798509 2022-08-16 08:54:01.247634 2022-08-16 08:54:01.480732   

                   duration  params_s1  params_s2  params_s3     state  
1838 0 days 00:00:00.228642   0.661639   0.179516   0.000020  COMPLETE  
3105 0 days 00:00:00.243010   0.737003   0.200296   0.000014  COMPLETE  
2179 0 days 00:00:00.218418   0.820708   0.222565   0.000208  COMPLETE  
1089 0 days 00:00:00.215087   0.855599   0.231859   0.000235  COMPLETE  
2033 0 days 00:00:00.233098   0.838676   0.228714   0.000140  COMPLETE

In [13]:
study.best_params

{'s1': 0.6616385347180365,
 's2': 0.1795158215697932,
 's3': 2.0013989295381495e-05}

In [15]:
prefixes = ['oof_v2_dpv25fold_seed42', 'oof_v2_5fold_seed98', 
            'oof_v4_dpv25fold_seed42', 'oof_lgbm_baseline_5fold_seed42',
            'cboof_v2_dpv25fold_seed802', 'oof_v1_dpv35fold_seed42'
           ]

valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_1 = trial.suggest_uniform('s1', 0.0, 1.0)
    p_2 = trial.suggest_uniform('s2', 0.0, 1.0)
    p_3 = trial.suggest_uniform('s3', 0.0, 1.0)
    p_4 = trial.suggest_uniform('s4', 0.0, 1.0)
    p_5 = trial.suggest_uniform('s5', 0.0, 1.0)
    p_6 = trial.suggest_uniform('s6', 0.0, 1.0)
    
    y = valids[0]['target'].values
    
    a_1 = valids[0].drop(['target','customer_ID'], axis=1).values
    a_2 = valids[1].drop(['target','customer_ID'], axis=1).values
    a_3 = valids[2].drop(['target','customer_ID'], axis=1).values
    a_4 = valids[3].drop(['target','customer_ID'], axis=1).values
    a_5 = valids[4].drop(['target','customer_ID'], axis=1).values
    a_6 = valids[5].drop(['target','customer_ID'], axis=1).values
    
#     a_4 = a_5 = a_6 = 0
#     p_4 = p_5 = p_6 = 0
    
    a = (p_1 * a_1 + p_2 * a_2 + p_3 * a_3 + p_4 * a_4 + p_5 * a_5 + p_6 * a_6)
    b = a.reshape(len(a_1))/(p_1+p_2+p_3+p_4+p_5+p_6)

    return amex_metric(y, np.clip(b, 0, 1))

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=1800)

[I 2022-08-16 09:00:53,800] A new study created in memory with name: no-name-e123a3ea-fc5b-47b5-8211-7d2078392221
[I 2022-08-16 09:00:53,951] Trial 0 finished with value: 0.7716728017965696 and parameters: {'s1': 0.15518852284781948, 's2': 0.9113901856557619, 's3': 0.3487182585054375, 's4': 0.5266618064218691, 's5': 0.5847997840821336, 's6': 0.7553166587230359}. Best is trial 0 with value: 0.7716728017965696.
[I 2022-08-16 09:00:54,093] Trial 1 finished with value: 0.7787126447810612 and parameters: {'s1': 0.7258259107920879, 's2': 0.5995072997501859, 's3': 0.6673089404988386, 's4': 0.08363881038817134, 's5': 0.40335292769646436, 's6': 0.9324474865743518}. Best is trial 1 with value: 0.7787126447810612.
[I 2022-08-16 09:00:54,222] Trial 2 finished with value: 0.7379970784284675 and parameters: {'s1': 0.20197983651109275, 's2': 0.007346480011791279, 's3': 0.12119259255698467, 's4': 0.08664209815397572, 's5': 0.6323779015105713, 's6': 0.017055971537562442}. Best is trial 1 with value: 0.

[I 2022-08-16 09:00:58,269] Trial 26 finished with value: 0.7850640616235539 and parameters: {'s1': 0.39335838063886436, 's2': 0.04596754637705014, 's3': 0.29907797530962965, 's4': 0.5531164997726814, 's5': 0.13676783495676442, 's6': 0.442918109535409}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:00:58,428] Trial 27 finished with value: 0.7748942598595281 and parameters: {'s1': 0.4859401144473655, 's2': 0.2822924940316664, 's3': 0.005804684877522337, 's4': 0.4658240020286346, 's5': 0.26884585202481703, 's6': 0.2362221819692281}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:00:58,603] Trial 28 finished with value: 0.7903638896765137 and parameters: {'s1': 0.6848530150687216, 's2': 0.1291057604804015, 's3': 0.18613424968534822, 's4': 0.3713760613697421, 's5': 0.08173624925778367, 's6': 0.6131903426864184}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:00:58,760] Trial 29 finished with value: 0.7664674940222711 and parameters:

[I 2022-08-16 09:01:02,835] Trial 52 finished with value: 0.7971804201194301 and parameters: {'s1': 0.3092191461929318, 's2': 0.9938241933420379, 's3': 0.4782226359666545, 's4': 0.05101587025872748, 's5': 0.006746335904840438, 's6': 0.401174507642118}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:02,990] Trial 53 finished with value: 0.7526904048008847 and parameters: {'s1': 0.20031644251804814, 's2': 0.051463330446326006, 's3': 0.4295259225203436, 's4': 0.004255434654252002, 's5': 0.6839341935461444, 's6': 0.4524149595832455}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:03,152] Trial 54 finished with value: 0.781381813313567 and parameters: {'s1': 0.2659076931647935, 's2': 0.0005856092317971587, 's3': 0.3946144661802178, 's4': 0.12450341344189109, 's5': 0.108003134348513, 's6': 0.15114363950206378}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:03,324] Trial 55 finished with value: 0.792932541983578 and parameters

[I 2022-08-16 09:01:07,489] Trial 78 finished with value: 0.7980352150158061 and parameters: {'s1': 0.5768416723003434, 's2': 0.7829340076085486, 's3': 0.27137566791902656, 's4': 0.3543118599643677, 's5': 0.003506055407148754, 's6': 0.07288292194049506}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:07,655] Trial 79 finished with value: 0.7864982624590704 and parameters: {'s1': 0.5745376905705757, 's2': 0.7944078938151428, 's3': 0.2643911213341702, 's4': 0.3487228358884819, 's5': 0.14091775875422022, 's6': 0.07219860666851237}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:07,826] Trial 80 finished with value: 0.7922665602378995 and parameters: {'s1': 0.7098719931070366, 's2': 0.7288683013112053, 's3': 0.1995170542413347, 's4': 0.17331177549720136, 's5': 0.05768837920325563, 's6': 0.22500222606437026}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:08,020] Trial 81 finished with value: 0.7980272847050254 and parameters

[I 2022-08-16 09:01:12,337] Trial 104 finished with value: 0.7979151476335649 and parameters: {'s1': 0.42535236679131033, 's2': 0.8858816541645406, 's3': 0.406001905083974, 's4': 0.10184571079331979, 's5': 0.001225289509072791, 's6': 0.8982302611945987}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:12,516] Trial 105 finished with value: 0.7924779738871108 and parameters: {'s1': 0.6263157089733821, 's2': 0.8192083674144707, 's3': 0.342195882638234, 's4': 0.15316709700243636, 's5': 0.06889884093929872, 's6': 0.9049122932057827}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:12,699] Trial 106 finished with value: 0.7944803824268823 and parameters: {'s1': 0.7093072860849844, 's2': 0.926943372858772, 's3': 0.2845888317440878, 's4': 0.20265791800161884, 's5': 0.04462527859362792, 's6': 0.8045387134116704}. Best is trial 22 with value: 0.7985611534507057.
[I 2022-08-16 09:01:12,886] Trial 107 finished with value: 0.796332880143842 and parameters:

...

[I 2022-08-16 09:30:11,475] Trial 6526 finished with value: 0.7986414331189131 and parameters: {'s1': 0.9692216673015591, 's2': 0.4735933019116438, 's3': 0.5213981023364473, 's4': 0.46938300983802794, 's5': 0.00017841085873498578, 's6': 0.3737184011286794}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:11,844] Trial 6527 finished with value: 0.7986947791225846 and parameters: {'s1': 0.9438638508852077, 's2': 0.4179993205100845, 's3': 0.8045409106710776, 's4': 0.39350511560687174, 's5': 0.00046019298252499035, 's6': 0.355953094863543}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:12,174] Trial 6528 finished with value: 0.796018857205393 and parameters: {'s1': 0.9996032548028783, 's2': 0.45286564452112016, 's3': 0.8329168268470872, 's4': 0.443585464573121, 's5': 0.03274484363819224, 's6': 0.40430415792594243}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:12,553] Trial 6529 finished with value: 0.7986710194995069

[I 2022-08-16 09:30:20,715] Trial 6552 finished with value: 0.7986911895527882 and parameters: {'s1': 0.9663312952742268, 's2': 0.3623530811034668, 's3': 0.7819241824941117, 's4': 0.27425074339747446, 's5': 0.0007965996766907161, 's6': 0.40390046095169974}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:21,083] Trial 6553 finished with value: 0.7987424607498633 and parameters: {'s1': 0.9134658056899916, 's2': 0.43128554286041904, 's3': 0.8324277282974274, 's4': 0.38731334765695397, 's5': 0.0008859702471933765, 's6': 0.4494616431665388}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:21,466] Trial 6554 finished with value: 0.7987307097824227 and parameters: {'s1': 0.9837451483814172, 's2': 0.4694032630952307, 's3': 0.8027958010552947, 's4': 0.42783346340313755, 's5': 0.00016239171693774478, 's6': 0.37125688194264245}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:21,829] Trial 6555 finished with value: 0.7957438688

[I 2022-08-16 09:30:30,081] Trial 6578 finished with value: 0.7987566144189707 and parameters: {'s1': 0.9576855616428251, 's2': 0.3895982217415555, 's3': 0.7674210202893318, 's4': 0.4723862836172894, 's5': 0.0009070222551669263, 's6': 0.42488580212433386}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:30,448] Trial 6579 finished with value: 0.7945756694416044 and parameters: {'s1': 0.8702056061102272, 's2': 0.44176915925031734, 's3': 0.7873546677183217, 's4': 0.4474376769118864, 's5': 0.04947974673795897, 's6': 0.36777307340235604}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:30,811] Trial 6580 finished with value: 0.7986119096023672 and parameters: {'s1': 0.9039327892040527, 's2': 0.4216786261741075, 's3': 0.824103291077312, 's4': 0.48938966261053113, 's5': 0.0010829612804280062, 's6': 0.303904108204126}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:31,174] Trial 6581 finished with value: 0.7987552930858388 

[I 2022-08-16 09:30:39,131] Trial 6604 finished with value: 0.7909351817925317 and parameters: {'s1': 0.9823254631840341, 's2': 0.4728218457437142, 's3': 0.49276792668252595, 's4': 0.5034861023299955, 's5': 0.10829263578414906, 's6': 0.36312953884961263}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:39,517] Trial 6605 finished with value: 0.7988138889495344 and parameters: {'s1': 0.9500364973205325, 's2': 0.4268719082391561, 's3': 0.8083018935817464, 's4': 0.44248627967610127, 's5': 0.0001251979995412324, 's6': 0.40568396377920685}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:39,857] Trial 6606 finished with value: 0.79698839462644 and parameters: {'s1': 0.9734470725786327, 's2': 0.45639551164109954, 's3': 0.7774560454263197, 's4': 0.48546613338646394, 's5': 0.020077447532625196, 's6': 0.350007466192394}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:40,197] Trial 6607 finished with value: 0.7972745327257067 

[I 2022-08-16 09:30:48,359] Trial 6630 finished with value: 0.7987303739647131 and parameters: {'s1': 0.9539319958340884, 's2': 0.45610338613562923, 's3': 0.7886507206469953, 's4': 0.43227952442126677, 's5': 0.0005818810347249371, 's6': 0.3979813532907993}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:48,772] Trial 6631 finished with value: 0.7987610263997051 and parameters: {'s1': 0.9833171118766443, 's2': 0.4288605083830845, 's3': 0.7698631405427806, 's4': 0.4646640860530591, 's5': 0.00010759645374342129, 's6': 0.38060950162653595}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:49,103] Trial 6632 finished with value: 0.7948513351794169 and parameters: {'s1': 0.9998204380994798, 's2': 0.4495836525367401, 's3': 0.8037117849433322, 's4': 0.5297465403624388, 's5': 0.05061972577647195, 's6': 0.399974228503072}. Best is trial 2342 with value: 0.7988405865219368.
[I 2022-08-16 09:30:49,443] Trial 6633 finished with value: 0.7958784833297647

In [16]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

number     value             datetime_start          datetime_complete  \
2342    2342  0.798841 2022-08-16 09:09:28.836861 2022-08-16 09:09:29.110563   
752      752  0.798835 2022-08-16 09:03:24.177846 2022-08-16 09:03:24.416005   
2678    2678  0.798834 2022-08-16 09:10:52.610436 2022-08-16 09:10:52.880809   
3973    3973  0.798833 2022-08-16 09:16:40.731024 2022-08-16 09:16:41.046392   
1626    1626  0.798830 2022-08-16 09:06:37.954922 2022-08-16 09:06:38.213042   

                   duration  params_s1  params_s2  params_s3  params_s4  \
2342 0 days 00:00:00.273702   0.977567   0.436344   0.773386   0.466529   
752  0 days 00:00:00.238159   0.982395   0.446221   0.798057   0.457557   
2678 0 days 00:00:00.270373   0.925320   0.372031   0.825991   0.422945   
3973 0 days 00:00:00.315368   0.999618   0.437059   0.783133   0.421318   
1626 0 days 00:00:00.258120   0.985868   0.442366   0.804630   0.462394   

      params_s5  params_s6     state  
2342   0.000063   0.403057  COMPLETE  
752    0.000221   0.418551  COMPLETE  
2678   0.000639   0.431939  COMPLETE  
3973   0.000359   0.464627  COMPLETE  
1626   0.000385   0.425011  COMPLETE

In [17]:
study.best_params

{'s1': 0.977567146304494,
 's2': 0.43634437929597414,
 's3': 0.7733860471719263,
 's4': 0.46652947925564303,
 's5': 6.290712170104372e-05,
 's6': 0.4030571518829691}

In [14]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

number     value             datetime_start          datetime_complete  \
618      618  0.799251 2022-08-15 21:50:04.634696 2022-08-15 21:50:04.871509   
1346    1346  0.799247 2022-08-15 21:53:10.383959 2022-08-15 21:53:10.637776   
5923    5923  0.799246 2022-08-15 22:17:23.608635 2022-08-15 22:17:23.970923   
6597    6597  0.799245 2022-08-15 22:21:42.664067 2022-08-15 22:21:43.042841   
2215    2215  0.799231 2022-08-15 21:57:07.695848 2022-08-15 21:57:07.969981   

                   duration  params_s1  params_s2  params_s3  params_s4  \
618  0 days 00:00:00.236813   0.899886   0.414343   0.962164   0.213392   
1346 0 days 00:00:00.253817   0.890166   0.411717   0.962403   0.208501   
5923 0 days 00:00:00.362288   0.924683   0.404875   0.979387   0.196034   
6597 0 days 00:00:00.378774   0.891237   0.394835   0.935200   0.217902   
2215 0 days 00:00:00.274133   0.924600   0.413037   0.998799   0.246682   

      params_s5  params_s6     state  
618    0.163728   0.268429  COMPLETE  
1346   0.176261   0.264168  COMPLETE  
5923   0.170853   0.277372  COMPLETE  
6597   0.172625   0.270306  COMPLETE  
2215   0.174446   0.279680  COMPLETE

In [15]:
study.best_params

{'s1': 0.8998860488225253,
 's2': 0.41434303527969907,
 's3': 0.9621639104415628,
 's4': 0.21339238932566207,
 's5': 0.16372836981554906,
 's6': 0.2684292708571563}

In [30]:
# best_params = {'s1': 0.8998860488225253,
#  's2': 0.41434303527969907,
#  's3': 0.9621639104415628,
#  's4': 0.21339238932566207,
#  's5': 0.16372836981554906,
#  's6': 0.2684292708571563}

p_1 = study.best_params['s1']
p_2 = study.best_params['s2']
p_3 = study.best_params['s3']
p_4 = study.best_params['s4']
p_5 = study.best_params['s5']
p_6 = study.best_params['s6']

# p_1 = best_params['s1']
# p_2 = best_params['s2']
# p_3 = best_params['s3']
# p_4 = best_params['s4']
# p_5 = best_params['s5']
# p_6 = best_params['s6']

a_1 = valids[0].drop(['customer_ID','target'], axis=1).values
a_2 = valids[1].drop(['customer_ID','target'], axis=1).values
a_3 = valids[2].drop(['customer_ID','target'], axis=1).values
a_4 = valids[3].drop(['customer_ID','target'], axis=1).values
a_5 = valids[4].drop(['customer_ID','target'], axis=1).values
a_6 = valids[5].drop(['customer_ID','target'], axis=1).values

# a_4 = a_5 = a_6 = 0
# p_4 = p_5 = p_6 = 0

# a = (p_1 * a_1 + p_2 * a_2 + p_3 * a_3 + p_4 * a_4 + p_5 * a_5 + p_6 * a_6)
# b = a.reshape(len(a_1))/(p_1+p_2+p_3+p_4+p_5+p_6)

a = (p_1 * a_1 + p_2 * a_2 + p_3 * a_3 + p_4 * a_4 + p_5 * a_5 + p_6 * a_6)
b = a.reshape(len(a_1))/(p_1+p_2+p_3+p_4+p_5+p_6)

    
val = valids[0][['customer_ID','target']].reset_index(drop=True)
val['prediction'] = np.clip(b, 0, 1)

for i in range(1, 14):
    lthan = amex_metric(val[val.customer_ID.isin(x[x['nunique']<=i].index.tolist())]['target'], 
            val[val.customer_ID.isin(x[x['nunique']<=i].index.tolist())]['prediction'])
    if i <13:
        gthan = amex_metric(val[val.customer_ID.isin(x[x['nunique']>i].index.tolist())]['target'], 
                val[val.customer_ID.isin(x[x['nunique']>i].index.tolist())]['prediction'])
    else: gthan = -1
    print(f'Nunique {i}: less-than score {lthan}, greater-than score{gthan}')
    
# Nunique 12: less-than score 0.6719086819329969, greater-than score0.8231005500411588
# Nunique 13: less-than score 0.7975376272588612, greater-than score-1

# Nunique 1: less-than score 0.6282347257253893, greater-than score0.8013327249905122
# Nunique 2: less-than score 0.6291073020364186, greater-than score0.8038608921024923
# Nunique 3: less-than score 0.6375589957778394, greater-than score0.8059978514986843
# Nunique 4: less-than score 0.6290729435497501, greater-than score0.8083890342687381
# Nunique 5: less-than score 0.6359608308411092, greater-than score0.809998018139306
# Nunique 6: less-than score 0.6429565062923831, greater-than score0.8116000827307112
# Nunique 7: less-than score 0.6477489999414328, greater-than score0.8133990539626946
# Nunique 8: less-than score 0.6524150724998354, greater-than score0.8153934471720639
# Nunique 9: less-than score 0.6598261793090174, greater-than score0.8171477286129305
# Nunique 10: less-than score 0.662020889037561, greater-than score0.8195831581564974
# Nunique 11: less-than score 0.6656850019771834, greater-than score0.8211871344155206
# Nunique 12: less-than score 0.6741658583385963, greater-than score0.8235210887515862
# Nunique 13: less-than score 0.7992506454997461, greater-than score-1

Nunique 1: less-than score 0.6313064465312965, greater-than score0.8008629903420148
Nunique 2: less-than score 0.6284167647777401, greater-than score0.8034007339670597
Nunique 3: less-than score 0.6362153117382183, greater-than score0.805456041935799
Nunique 4: less-than score 0.6308069620282215, greater-than score0.8078055288689621
Nunique 5: less-than score 0.6368233095600117, greater-than score0.8093588857520551
Nunique 6: less-than score 0.644152372690793, greater-than score0.8114369236955616
Nunique 7: less-than score 0.648013130876318, greater-than score0.8132548427459617
Nunique 8: less-than score 0.6526806719772716, greater-than score0.81549037207099
Nunique 9: less-than score 0.6587458805843741, greater-than score0.8172270833355375
Nunique 10: less-than score 0.6625792732065214, greater-than score0.8194718918646384
Nunique 11: less-than score 0.6652555021825481, greater-than score0.8212846891623571
Nunique 12: less-than score 0.67455884655262, greater-than score0.8237436431539

In [32]:
prefixes = ['lgb_v2_dpv25fold_seed42', 'lgb_v2_5fold_seed98', 
            'lgb_v4_dpv25fold_seed42', 'test_lgbm_baseline_5fold_seed42',
            'cb_v2_dpv25fold_seed802', 'lgb_v1_dpv35fold_seed42'
           ]

submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

best_params = {'s1': 0.977567146304494,
 's2': 0.43634437929597414,
 's3': 0.7733860471719263,
 's4': 0.46652947925564303,
 's5': 6.290712170104372e-05,
 's6': 0.4030571518829691}

p_1 = best_params['s1']
p_2 = best_params['s2']
p_3 = best_params['s3']
p_4 = best_params['s4']
p_5 = best_params['s5']
p_6 = best_params['s6']
# p_7 = best_params['s7']

a_1 = submissions[0].drop(['customer_ID'], axis=1).values
a_2 = submissions[1].drop(['customer_ID'], axis=1).values
a_3 = submissions[2].drop(['customer_ID'], axis=1).values
a_4 = submissions[3].drop(['customer_ID'], axis=1).values
a_5 = submissions[4].drop(['customer_ID'], axis=1).values
a_6 = submissions[5].drop(['customer_ID'], axis=1).values
# a_7 = submissions[6].drop(['customer_ID'], axis=1).values


a = (p_1 * a_1 + p_2 * a_2 + p_3 * a_3 + p_4 * a_4 + p_5 * a_5 + p_6 * a_6)
b = a.reshape(len(a_1))/(p_1+p_2+p_3+p_4+p_5+p_6)

sub = submissions[0][['customer_ID']].reset_index(drop=True)

sub['prediction'] = np.clip(b, 0, 1)

sub.to_csv('blend_6_cv_79884.csv', index=False)

' '.join(prefixes)

'lgb_v2_dpv25fold_seed42 lgb_v2_5fold_seed98 lgb_v4_dpv25fold_seed42 test_lgbm_baseline_5fold_seed42 cb_v2_dpv25fold_seed802 lgb_v1_dpv35fold_seed42'

In [33]:
!kaggle competitions submit -c amex-default-prediction -f blend_6_cv_79884.csv -m "blend 6 models CV 79884 \
lgb_v2_dpv25fold_seed42 lgb_v2_5fold_seed98 lgb_v4_dpv25fold_seed42 test_lgbm_baseline_5fold_seed42 cb_v2_dpv25fold_seed802 lgb_v1_dpv35fold_seed42 \
"
! rm blend_6_cv_79884.csv

100%|██████████████████████████████████████| 75.3M/75.3M [00:06<00:00, 12.8MB/s]
Successfully submitted to American Express - Default Prediction